In [3]:
import pandas as pd
import os
import numpy as np
import time
from multiprocessing.dummy import Pool as ThreadPool
import datetime
import pickle

calculated_features = r"X:\CIS-PD Study\MJFF Curation\all pre-visit calc features"
features = r"X:\CIS-PD Study\\Home WACC features"
destination = r"X:\CIS-PD Study\\Home WACC features + Gait"
files = os.listdir(calculated_features)

['142578 2017-09-18.csv', '142616 2018-02-05.csv', '142557 2017-06-29.csv', '142616 2017-10-03.csv', '142620 2017-10-30.csv', '142568 2017-09-18.csv', '142558 2017-07-14.csv', '142609 2017-09-01.csv', '142612 2017-09-06.csv', '142568 2018-02-28.csv', '142615 2017-10-05.csv', '142620 2017-08-10.csv', '142593 2017-11-06.csv', '142601 2017-07-13.csv', '142603 2017-10-13.csv', '142615 2017-07-21.csv', '142558 2017-09-21.csv', '142595 2017-10-12.csv', '142621 2017-10-23.csv', '142608 2017-11-09.csv', '142580 2017-08-01.csv', '142577 2017-11-30.csv', '142603 2017-08-07.csv', '142575 2017-08-29.csv', '142608 2017-09-11.csv', '142619 2017-08-10.csv', '142577 2017-10-11.csv', '142570 2017-07-13.csv', '142604 2017-10-02.csv', '142623 2017-08-03.csv', '142585 2017-09-20.csv', '142583 2017-10-13.csv', '142616 2017-08-01.csv', '142623 2017-10-12.csv', '142604 2017-08-14.csv', '142566 2017-08-09.csv', '142561 2017-07-03.csv', '142619 2017-07-21.csv', '142605 2017-10-11.csv', '142559 2017-07-20.csv',

In [5]:
def drop_Movements_Level(file):
    print(file)
    cf = pd.read_csv(calculated_features + "\\" + file)
    for index, row in cf.iterrows():
        if (row['measurement_id'] == 11):
            cf = cf.drop(index)
    cf.to_csv(calculated_features + "\\" + file)

In [6]:
def drop_extra_index_columns(file):
    print(file)
    cf = pd.read_csv(calculated_features + "\\" + file)
    cols = list(cf.columns.values)
    for col in cols:
        if "Unnamed" in col:
            cf = cf.drop(col, axis = 1)
    cf.to_csv(calculated_features + "\\" + file)

In [7]:
def find_corresponding_cf_data(tm, cf):
    tm = tm - datetime.timedelta(minutes=0,
                             seconds=tm.second % 5,
                             microseconds=tm.microsecond)
    tm = tm.strftime('%Y-%m-%d %H:%M:%S.000')
    return [cf.loc[(13,tm)].value,cf.loc[(10,tm)].value,cf.loc[(7,tm)].value]
                
                

In [8]:
### take the features in file and pair them with the corresponding Gait, Activity Level, and TremorScore data ###
def merge_file(file):
    t1 = time.time()
    print("1:")
    print(file)
    cf = pd.read_csv(calculated_features + "\\" + file,dtype={'measurement_id': int, 'value': float})
    f = pd.read_pickle(features + "\\features " + file[0:-3] + "pkl")

    f['timestamp'] = pd.to_datetime(f['timestamp'])

    Gait = cf[cf.measurement_id==13][['value','timestamp']]
    ActLevel = cf[cf.measurement_id==10][['value','timestamp']]
    TremorScore = cf[cf.measurement_id==7][['value','timestamp']]

    Gait = Gait.rename(columns={'value':'Gait'})
    ActLevel = ActLevel.rename(columns={'value':'Activity Level'})
    TremorScore = TremorScore.rename(columns={'value':'TremorScore'})

    Gait = pd.merge(Gait,ActLevel,on='timestamp')
    Gait = pd.merge(Gait,TremorScore, on='timestamp')
    Gait.timestamp = pd.to_datetime(Gait.timestamp)
    f = pd.merge(f,Gait, on='timestamp')

    with open(destination + "\\features+gait " + file[0:-3] + 'pkl','wb') as savefile:
        pickle.dump(f, savefile)
        

    t_elapsed = ((time.time() - t1)/60.0)
    print(t_elapsed.__str__() + " mins")

In [9]:
for file in files:
    merge_file(file)

1:
142578 2017-09-18.csv
0.001659997304280599 mins
1:
142616 2018-02-05.csv
0.0018200039863586425 mins
1:
142557 2017-06-29.csv
0.005733335018157959 mins
1:
142616 2017-10-03.csv
0.0031199971834818523 mins
1:
142620 2017-10-30.csv
0.007800002892812093 mins
1:
142568 2017-09-18.csv
0.005130000909169515 mins
1:
142558 2017-07-14.csv


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2903: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


0.020403329531351724 mins
1:
142609 2017-09-01.csv


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2903: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


0.012973332405090332 mins
1:
142612 2017-09-06.csv
0.009930002689361572 mins
1:
142568 2018-02-28.csv


...

0.023329997062683107 mins
1:
142602 2017-08-03.csv
0.013233331839243571 mins
1:
142595 2018-01-10.csv


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2903: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


0.013036664326985676 mins
1:
142621 2017-07-21.csv


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2903: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


0.01972333590189616 mins


In [10]:
# pool = ThreadPool()
# pool.map(merge_file, files)
# pool.close()
# pool.join()